<a href="https://colab.research.google.com/github/Sillem/suicide_study/blob/main/Projekt_EP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
from math import fabs
from sklearn.preprocessing import StandardScaler 

Ustalenie ścieżki do folderu z danymi.

In [2]:
path = r'/content/drive/MyDrive/Projekt EKONOMETRIA PRZESTRZENNA/dane'

Tworzenie słownika z zapisanymi ramkami danych z każdego pliku i zapisywanie ich z podanej ściezki


In [3]:
data = {}

for root, dirs, files in os.walk(path, topdown=False):
   for name in files:
      data[name.replace('.xlsx', '')] = pd.read_excel(os.path.join(root, name)).rename(columns={'Unnamed: 0':'wojewodztwo'})
      data[name.replace('.xlsx', '')]['wojewodztwo'] = data[name.replace('.xlsx', '')]['wojewodztwo'].apply(lambda x: x.replace('WOJ. ', '').rstrip().upper())
      data[name.replace('.xlsx', '')] = data[name.replace('.xlsx', '')].set_index('wojewodztwo').rename(columns={'2018':2018, '2021':2021})

Wyliczanie współczynników zmienności dla każdej zmiennej:

In [4]:
k = {'Współczynnik zmienności':{}}
for key, value in data.items():
  k['Współczynnik zmienności'][key] = f"{round((value.describe().loc['std']/value.describe().loc['mean']).mean() * 100, 2)}%" 

pd.DataFrame(k).sort_values(by='Współczynnik zmienności', ascending=False)

,Współczynnik zmienności
zatrudnieni_zagrozenie,94.37%
rozwody,58.36%
katolicyzm,58.04%
dochody_budzetow_ogolem,54.45%
liczba_samobojstw_na_10_tys,31.93%
stopa_bezrobocia,26.62%
liczba_psychiatrow_na_10_tys,15.94%
osoby_z_wyksztalceniem_podstawowym,15.8%
procent_mezczyzn_w_wojewodztwach,0.78%


Usuwam z badanych zmiennych zmienną procent_mezczyzn_w_wojewodztwach z uwagi na zbyt małą zmienność.

In [5]:
del(data['procent_mezczyzn_w_wojewodztwach'])

Badanie skorelowania zmiennych dla każdego roku. Ale najpierw zbudowanie kompletnego zestawu danych dla każdego roku.

In [6]:
yearly_data = {2010:pd.DataFrame({}), 2014:pd.DataFrame({}), 2018:pd.DataFrame({}), 2021:pd.DataFrame({})}
for key, value in data.items():
  for year in yearly_data.keys():
    yearly_data[year] = pd.concat([yearly_data[year], value[year].rename(key)], axis=1)

((yearly_data[2021].corr() + yearly_data[2018].corr() + yearly_data[2014].corr() + yearly_data[2010].corr())/4).applymap(lambda x: round(x, 3) if fabs(x)>0.8 else 0)

del(data['katolicyzm'])

## PORZĄDKOWANIE LINIOWE

Tworzę kopię danych odnośnie samobójstw

In [7]:
y = data['liczba_samobojstw_na_10_tys']

Usuwam zmienną związaną z katolicyzmem i rozwodami. Usuwam też zmienną objaśniającą.

In [8]:
# del(data['katolicyzm'])
del(data['rozwody'])
del(data['liczba_samobojstw_na_10_tys'])

Sprawdzam korelacje teraz po usunięciu zmiennych.


In [9]:
yearly_data = {2010:pd.DataFrame({}), 2014:pd.DataFrame({}), 2018:pd.DataFrame({}), 2021:pd.DataFrame({})}
for key, value in data.items():
  for year in yearly_data.keys():
    yearly_data[year] = pd.concat([yearly_data[year], value[year].rename(key)], axis=1)

((yearly_data[2021].corr() + yearly_data[2018].corr() + yearly_data[2014].corr() + yearly_data[2010].corr())/4).applymap(lambda x: round(x, 3) if fabs(x)>0.6 else 0)

,dochody_budzetow_ogolem,liczba_psychiatrow_na_10_tys,stopa_bezrobocia,zatrudnieni_zagrozenie,osoby_z_wyksztalceniem_podstawowym
dochody_budzetow_ogolem,1.0,0.0,0.000,0.000,0.000
liczba_psychiatrow_na_10_tys,0.0,1.0,0.000,0.000,0.000
stopa_bezrobocia,0.0,0.0,1.000,-0.612,0.682
zatrudnieni_zagrozenie,0.0,0.0,-0.612,1.000,-0.651
osoby_z_wyksztalceniem_podstawowym,0.0,0.0,0.682,-0.651,1.000


Standaryzacja zmiennych

In [10]:
def standaryzacja_zmiennych(df):
  scaler = StandardScaler()
  return pd.DataFrame(data=scaler.fit_transform(df.values), columns=df.columns, index=df.index)


Funkcja budująca ranking topsis

In [11]:
# tutaj wektor stym_czy_destym będzie definowany jako wektor złożony ze znaków 's' jeżeli kolumna oznacza stymulante
# 'd' jeżeli destymulantę
# dla przykładu dla zemiennych w kolejności stymulanta, destymulanta, stymulanta wektor będzie wyglądał tak ['s', 'd', 's']
def topsis(df, stym_czy_destym):

  def standaryzacja_topsis_zmiennych(df):
    df_copy = df.copy()
    for col in df_copy.columns:
      sqrt_sum_sq = sum(df_copy[col].apply(lambda x: x**2))**0.5
      df_copy[col] = df_copy[col].apply(lambda x: x/sqrt_sum_sq)
    return df_copy
  df = standaryzacja_topsis_zmiennych(df)

  wzorzec = []
  antywzorzec = []
  if df.columns.shape[0] != len(stym_czy_destym):
    raise ValueError("Liczba elementów wektora określającego charakter zmiennych i liczba kolumn są różne.")
  for numer, col in enumerate(df.columns):
    if stym_czy_destym[numer] == 's':
      wzorzec.append(max(df[col]))
      antywzorzec.append(min(df[col]))
    elif stym_czy_destym[numer] == 'd':
      wzorzec.append(min(df[col]))
      antywzorzec.append(max(df[col]))
    else:
      raise ValueError("Podano inny parametr w wektorze określającym charakter zmiennych niż 's' i 'd'.")
  wzorzec, antywzorzec = np.array(wzorzec), np.array(antywzorzec)
  kopia_operacyjna = df.copy()
  odl_od_wzorca = np.diag(np.matmul((kopia_operacyjna - wzorzec).values, np.transpose((kopia_operacyjna - wzorzec).values)))**0.5
  odl_od_antywzorca = np.diag(np.matmul((kopia_operacyjna - antywzorzec).values, np.transpose((kopia_operacyjna - antywzorzec).values)))**0.5
  topsis = odl_od_antywzorca / (odl_od_wzorca + odl_od_antywzorca)
  ranks = np.argsort(np.argsort(-topsis)) + 1

  return pd.DataFrame(data={'TOPSIS':topsis, 'POZYCJA':ranks}, index=df.index)



Kod do metody sum standaryzowanych

In [12]:
def metoda_sum_std(df, stym_czy_destym):
  """
  Tutaj wymagana jest zewnętrzna funkcja standaryzowania zmiennych z innych komórek,
  nie umieściłem jej wewnątrz tej funkcji żeby móc z niej korzystać przy innych operacjach.
  Przekształcenie topsis umieściłem w funkcji, ponieważ jest ono poza nią bezużyteczne
  """
  df = standaryzacja_zmiennych(df)
  if df.columns.shape[0] != len(stym_czy_destym):
    raise ValueError("Liczba elementów wektora określającego charakter zmiennych i liczba kolumn są różne.")
  for numer, col in enumerate(df.columns):
    if stym_czy_destym[numer] == 's':
      continue
    elif stym_czy_destym[numer] == 'd':
      df[col] = -df[col] #dla destymulant dajemy przekształconą zmienną z minusem
    else:
      raise ValueError("Podano inny parametr w wektorze określającym charakter zmiennych niż 's' i 'd'.")
  Wi = df.transpose().mean()
  Wi_odj_min = Wi - min(Wi)
  Wi_std = Wi_odj_min / max(Wi_odj_min)
  ranks = np.argsort(np.argsort(-Wi_std)) + 1
  return pd.DataFrame(data={'METODA SUM STANDARYZOWANYCH':Wi_std, 'POZYCJA':ranks}, index=df.index)


# mss = metoda_sum_std(yearly_data[2021], ['d','d','s', 's', 's'])


Kod odpowiedzialny za liczenie rankingu SMR

In [44]:
def smr(df, stym_czy_destym):
  """
  Tutaj wymagana jest zewnętrzna funkcja standaryzowania zmiennych z innych komórek,
  nie umieściłem jej wewnątrz tej funkcji żeby móc z niej korzystać przy innych operacjach.
  Przekształcenie topsis umieściłem w funkcji, ponieważ jest ono poza nią bezużyteczne
  """
  df = standaryzacja_zmiennych(df)
  if df.columns.shape[0] != len(stym_czy_destym):
    raise ValueError("Liczba elementów wektora określającego charakter zmiennych i liczba kolumn są różne.")
  for numer, col in enumerate(df.columns):
    if stym_czy_destym[numer] == 's':
      continue
    elif stym_czy_destym[numer] == 'd':
      df[col] = -df[col] #dla destymulant dajemy przekształconą zmienną z minusem
    else:
      raise ValueError("Podano inny parametr w wektorze określającym charakter zmiennych niż 's' i 'd'.")
  wzorzec = []
  for col in df.columns:
    wzorzec.append(max(df[col]))
  wzorzec = np.array(wzorzec)
  kopia_operacyjna = df.copy()
  odl_od_wzorca = np.diag(np.matmul((kopia_operacyjna - wzorzec).values, np.transpose((kopia_operacyjna - wzorzec).values)))**0.5
  SMR = 1 - odl_od_wzorca/max(odl_od_wzorca)
  ranks = ranks = np.argsort(np.argsort(-SMR)) + 1
  return pd.DataFrame(data={'SMR':SMR, 'POZYCJA':ranks}, index=df.index)



Teraz przygotowuje kod powiedzialny za wyliczanie rankingu metodą BZW

In [43]:
def BZW(df, stym_czy_destym):
  """
  Tutaj wymagana jest zewnętrzna funkcja standaryzowania zmiennych z innych komórek,
  nie umieściłem jej wewnątrz tej funkcji żeby móc z niej korzystać przy innych operacjach.
  Przekształcenie topsis umieściłem w funkcji, ponieważ jest ono poza nią bezużyteczne
  """
  df = standaryzacja_zmiennych(df)
  if df.columns.shape[0] != len(stym_czy_destym):
    raise ValueError("Liczba elementów wektora określającego charakter zmiennych i liczba kolumn są różne.")
  for numer, col in enumerate(df.columns):
    if stym_czy_destym[numer] == 's':
      continue
    elif stym_czy_destym[numer] == 'd':
      df[col] = -df[col] #dla destymulant dajemy przekształconą zmienną z minusem
    else:
      raise ValueError("Podano inny parametr w wektorze określającym charakter zmiennych niż 's' i 'd'.")
  kopia_operacyjna = df.copy()
  for col in df.columns:
    kopia_operacyjna[col] = kopia_operacyjna[col] + fabs(min(kopia_operacyjna[col])) 
  max_bzw = sum(kopia_operacyjna.describe().loc['max'].values)
  BZW = kopia_operacyjna.sum(axis=1)/max_bzw
  ranks = ranks = np.argsort(np.argsort(-BZW)) + 1
  return pd.DataFrame(data={'BZW':BZW, 'POZYCJA':ranks}, index=df.index)

Teraz przygotowuje kod odpowiedzialny za wyliczanie rankingu metodą Kukuły

In [45]:
def metoda_kukuly(df, stym_czy_destym):

  def standaryzacja_kukula_zmiennych(df, stym_czy_destym):
    df_copy = df.copy()
    if df.columns.shape[0] != len(stym_czy_destym):
      raise ValueError("Liczba elementów wektora określającego charakter zmiennych i liczba kolumn są różne.")
    for numer, col in enumerate(df.columns):
      if stym_czy_destym[numer] == 's':
        df_copy[col] = (df_copy[col] - min(df_copy[col]))/(max(df_copy[col]) - min(df_copy[col]))
      elif stym_czy_destym[numer] == 'd':
        df_copy[col] = (max(df_copy[col] - df_copy[col]))/(max(df_copy[col]) - min(df_copy[col]))
      else:
        raise ValueError("Podano inny parametr w wektorze określającym charakter zmiennych niż 's' i 'd'.")
    return df_copy
  df = standaryzacja_kukula_zmiennych(df, stym_czy_destym)

  Qi = df.mean(axis=1)
  ranks = np.argsort(np.argsort(-Qi)) + 1
  return pd.DataFrame(data={'METODA KUKUŁY':Qi, 'POZYCJA':ranks}, index=df.index)


kukula = metoda_kukuly(yearly_data[2021], ['d','d','s', 's', 's'])
kukula

,METODA KUKUŁY,POZYCJA
DOLNOŚLĄSKIE,0.137334,13
KUJAWSKO-POMORSKIE,0.264866,4
LUBELSKIE,0.292420,3
LUBUSKIE,0.118188,16
ŁÓDZKIE,0.224667,9
MAŁOPOLSKIE,0.141103,11
MAZOWIECKIE,0.121804,15
OPOLSKIE,0.140649,12
PODKARPACKIE,0.299886,2
PODLASKIE,0.256415,6


Tutaj przygotuje kod odpowiedzialny za wyliczanie rankingu metodą Dysonansów

In [46]:
def metoda_dysonansow(df, stym_czy_destym):

  def standaryzacja_metoda_dysonansow(df, stym_czy_destym):
    df_copy = df.copy()
    if df.columns.shape[0] != len(stym_czy_destym):
      raise ValueError("Liczba elementów wektora określającego charakter zmiennych i liczba kolumn są różne.")
    for numer, col in enumerate(df.columns):
      if stym_czy_destym[numer] == 's':
        df_copy[col] = df_copy[col]/max(df_copy[col])
      elif stym_czy_destym[numer] == 'd':
        df_copy[col] = min(df_copy[col])/df_copy[col]
      else:
        raise ValueError("Podano inny parametr w wektorze określającym charakter zmiennych niż 's' i 'd'.")
    return df_copy
  df = standaryzacja_metoda_dysonansow(df, stym_czy_destym)

  MD = df.mean(axis=1)
  ranks = np.argsort(np.argsort(-MD)) + 1
  return pd.DataFrame(data={'METODA DYSONANSÓW':MD, 'POZYCJA':ranks}, index=df.index)


dysonans = metoda_dysonansow(yearly_data[2021], ['d','d','s', 's', 's'])
dysonans

,METODA DYSONANSÓW,POZYCJA
DOLNOŚLĄSKIE,0.499083,14
KUJAWSKO-POMORSKIE,0.599622,7
LUBELSKIE,0.580755,9
LUBUSKIE,0.655636,4
ŁÓDZKIE,0.529982,12
MAŁOPOLSKIE,0.478702,15
MAZOWIECKIE,0.423906,16
OPOLSKIE,0.658526,3
PODKARPACKIE,0.646367,5
PODLASKIE,0.631621,6


Tutaj przygotuje kod odpowiedzialny za przygotowanie rankingów Metodą Sumy Rang


In [47]:
def metoda_suma_rang(df, stym_czy_destym):

  def standaryzacja_metoda_suma_rang(df, stym_czy_destym):
    df_copy = df.copy()
    if df.columns.shape[0] != len(stym_czy_destym):
      raise ValueError("Liczba elementów wektora określającego charakter zmiennych i liczba kolumn są różne.")
    for numer, col in enumerate(df.columns):
      if stym_czy_destym[numer] == 's':
        df_copy[col] = np.argsort(np.argsort(df_copy[col])) + 1
      elif stym_czy_destym[numer] == 'd':
        df_copy[col] = np.argsort(np.argsort(-df_copy[col])) + 1
      else:
        raise ValueError("Podano inny parametr w wektorze określającym charakter zmiennych niż 's' i 'd'.")
    return df_copy
  df = standaryzacja_metoda_suma_rang(df, stym_czy_destym)

  MSM = df.sum(axis=1)
  ranks = np.argsort(np.argsort(-MSM)) + 1
  return pd.DataFrame(data={'SUMA RANG':MSM, 'POZYCJA':ranks}, index=df.index)


suma_rang = metoda_suma_rang(yearly_data[2021], ['d','d','s', 's', 's'])
suma_rang

,SUMA RANG,POZYCJA
DOLNOŚLĄSKIE,30,14
KUJAWSKO-POMORSKIE,52,4
LUBELSKIE,43,8
LUBUSKIE,50,5
ŁÓDZKIE,40,10
MAŁOPOLSKIE,34,13
MAZOWIECKIE,21,16
OPOLSKIE,48,6
PODKARPACKIE,55,2
PODLASKIE,43,9


Tutaj przygotowuje kod odpowiedzialny za wyliczanie statystyki metody porównywania rankingów Kukuły.

In [18]:
def porownanie_rankingow(pozycje1, pozycje2):
  """
  Funkcja do której przekazujemy dwa obiekty typu pd.Series zawierające pozycje
  przyporządkowane do obiektów. NIE UPORZĄDKOWANE WEDŁUG POZYCJI! Można też podać
  DataFrame z rankingami zbudowanymi na modłę poprzednich przykładów.
  """
  differences = (pozycje1['POZYCJA'] - pozycje2['POZYCJA']).apply(lambda x: fabs(x))
  mianownik = differences.shape[0]**2 - 1 if differences.shape[0]**2 % 1 else differences.shape[0]**2
  return 1- 2*sum(differences)/mianownik




Tutaj definiuje listę metod używanych do użycia (lista funkcji)


In [19]:
metody = [topsis, metoda_sum_std, smr, BZW, metoda_kukuly, metoda_dysonansow, metoda_suma_rang]
rankingi = {x:{} for x in [2010, 2014, 2018, 2021]}
for rok in rankingi.keys():
  for metoda in metody:
    rankingi[rok][metoda.__name__] = metoda(yearly_data[rok], ['d','d','s', 's', 's'])

Tworzę metod porównywania rankingów

In [57]:
from random import randint
def optymalne_rankingi(lista_metod, rankingi):
  def macierz_porownan_rankingow(lista_metod, rankingi):
    macierz = np.identity(len(metody))
    for pierwszy_numer, metoda_pier in enumerate(lista_metod):
      for drugi_numer, metoda_drug in enumerate(lista_metod):
        if pierwszy_numer == drugi_numer: continue
        macierz[pierwszy_numer][drugi_numer] = porownanie_rankingow(rankingi[metoda_pier.__name__], rankingi[metoda_drug.__name__])

    return macierz

  macierz_wynikowa = macierz_porownan_rankingow(metody, rankingi)
  print(macierz_wynikowa)
  #pd.DataFrame(macierz_wynikowa).to_csv(f"{randint(1,100)}.csv")
  suma = 0
  for col in range(macierz_wynikowa.shape[1]):
    for row in range(macierz_wynikowa.shape[0]):
      if col > row: suma += macierz_wynikowa[row][col]

  srednie_podobienstwo = 2*suma/(len(metody)*(len(metody)-1))
  suma = 0
  for col in range(macierz_wynikowa.shape[1]):
    for row in range(macierz_wynikowa.shape[0]):
      if col > row: suma += fabs(macierz_wynikowa[row][col] - srednie_podobienstwo)

  odchylenie_cwiartkowe_podobienstwo = 2*suma/(len(metody)*(len(metody)-1))
  cutoff = srednie_podobienstwo + odchylenie_cwiartkowe_podobienstwo
  np.fill_diagonal(macierz_wynikowa, 0)
  avg_sim = pd.DataFrame({'średnie podobieństwo':{k.__name__:v for k,v in zip(lista_metod, macierz_wynikowa.mean(axis=1))}})
  avg_sim['średnie podobieństwo'] = avg_sim['średnie podobieństwo'].apply(lambda x: round(x, 3))
  best_ranking_index = np.argsort(macierz_wynikowa.mean(axis=1))[-3]
  
  return rankingi[lista_metod[best_ranking_index].__name__].sort_values(by='POZYCJA', ascending=True), avg_sim

pd.concat([optymalne_rankingi(metody, rankingi[rok])[1].rename(columns={'średnie podobieństwo':rok}) for rok in [2010, 2014, 2018, 2021]], axis=1)
pd.concat([optymalne_rankingi(metody, rankingi[rok])[0].reset_index().drop('SUMA RANG', axis=1).set_index('POZYCJA').rename(columns={'index':rok}) for rok in [2010, 2014, 2018, 2021]], axis=1)

[[1.       0.53125  0.640625 0.53125  0.3125   0.578125 0.53125 ]
 [0.53125  1.       0.75     1.       0.59375  0.78125  0.84375 ]
 [0.640625 0.75     1.       0.75     0.484375 0.625    0.765625]
 [0.53125  1.       0.75     1.       0.59375  0.78125  0.84375 ]
 [0.3125   0.59375  0.484375 0.59375  1.       0.640625 0.53125 ]
 [0.578125 0.78125  0.625    0.78125  0.640625 1.       0.734375]
 [0.53125  0.84375  0.765625 0.84375  0.53125  0.734375 1.      ]]
[[1.       0.546875 0.5      0.546875 0.296875 0.71875  0.5625  ]
 [0.546875 1.       0.78125  1.       0.546875 0.78125  0.8125  ]
 [0.5      0.78125  1.       0.78125  0.515625 0.609375 0.84375 ]
 [0.546875 1.       0.78125  1.       0.546875 0.78125  0.8125  ]
 [0.296875 0.546875 0.515625 0.546875 1.       0.46875  0.484375]
 [0.71875  0.78125  0.609375 0.78125  0.46875  1.       0.6875  ]
 [0.5625   0.8125   0.84375  0.8125   0.484375 0.6875   1.      ]]
[[1.       0.640625 0.5625   0.640625 0.484375 0.6875   0.59375 ]
 [0.6406

,2010,2014,2018,2021
POZYCJA,,,,
1,WARMIŃSKO-MAZURSKIE,PODKARPACKIE,WARMIŃSKO-MAZURSKIE,WARMIŃSKO-MAZURSKIE
2,LUBUSKIE,WARMIŃSKO-MAZURSKIE,LUBUSKIE,PODKARPACKIE
3,PODKARPACKIE,ŚWIĘTOKRZYSKIE,KUJAWSKO-POMORSKIE,ŚWIĘTOKRZYSKIE
4,ŚWIĘTOKRZYSKIE,LUBUSKIE,ŚWIĘTOKRZYSKIE,KUJAWSKO-POMORSKIE
5,OPOLSKIE,OPOLSKIE,PODKARPACKIE,LUBUSKIE
6,ZACHODNIOPOMORSKIE,KUJAWSKO-POMORSKIE,LUBELSKIE,OPOLSKIE
7,KUJAWSKO-POMORSKIE,ZACHODNIOPOMORSKIE,OPOLSKIE,ZACHODNIOPOMORSKIE
8,DOLNOŚLĄSKIE,POMORSKIE,ZACHODNIOPOMORSKIE,LUBELSKIE
9,PODLASKIE,WIELKOPOLSKIE,PODLASKIE,PODLASKIE


In [25]:
pd.concat([rankingi[rok]['BZW'].sort_values(by='POZYCJA', ascending=True).reset_index().set_index('POZYCJA').rename(columns={'index':rok}).drop('METODA SUM STANDARYZOWANYCH', axis=1) for rok in [2010, 2014, 2018, 2021]], axis=1)

,2010,2014,2018,2021
POZYCJA,,,,
1,WARMIŃSKO-MAZURSKIE,WARMIŃSKO-MAZURSKIE,WARMIŃSKO-MAZURSKIE,WARMIŃSKO-MAZURSKIE
2,LUBUSKIE,ŚWIĘTOKRZYSKIE,ŚWIĘTOKRZYSKIE,PODKARPACKIE
3,ŚWIĘTOKRZYSKIE,LUBUSKIE,LUBUSKIE,ŚWIĘTOKRZYSKIE
4,KUJAWSKO-POMORSKIE,OPOLSKIE,KUJAWSKO-POMORSKIE,KUJAWSKO-POMORSKIE
5,PODKARPACKIE,PODKARPACKIE,PODKARPACKIE,OPOLSKIE
6,OPOLSKIE,KUJAWSKO-POMORSKIE,LUBELSKIE,LUBUSKIE
7,ZACHODNIOPOMORSKIE,WIELKOPOLSKIE,OPOLSKIE,PODLASKIE
8,WIELKOPOLSKIE,ZACHODNIOPOMORSKIE,ZACHODNIOPOMORSKIE,LUBELSKIE
9,PODLASKIE,PODLASKIE,PODLASKIE,ZACHODNIOPOMORSKIE


Wyliczanie średniego podobieństwa rankingów.

In [22]:
ys = {}
for col in y.columns:
  ys[col] = pd.concat([(np.argsort(np.argsort(-y[col])) + 1), y[col]], axis=1)
  ys[col].columns = ['POZYCJA', 'L. SAMOBÓJSTW NA 10 TYS.']
  print(ys[col]['L. SAMOBÓJSTW NA 10 TYS.'].std() / ys[col]['L. SAMOBÓJSTW NA 10 TYS.'].mean() )
ys[2010]

0.2007890361898084
0.43905637940808717
0.28621257074993417
0.3510484672908047


,POZYCJA,L. SAMOBÓJSTW NA 10 TYS.
wojewodztwo,,
DOLNOŚLĄSKIE,15,0.837787
KUJAWSKO-POMORSKIE,7,1.217931
LUBELSKIE,4,1.321156
LUBUSKIE,6,1.277168
ŁÓDZKIE,5,1.314013
MAŁOPOLSKIE,8,1.058130
MAZOWIECKIE,9,1.041713
OPOLSKIE,12,0.911650
PODKARPACKIE,13,0.908774


Rankingi zbudowane na podstawie danych empirycznych

In [23]:
pd.concat([ys[rok].reset_index().set_index('POZYCJA').rename(columns={'wojewodztwo':rok}).drop('L. SAMOBÓJSTW NA 10 TYS.', axis=1) for rok in [2010, 2014, 2018, 2021]], axis=1).sort_index()

,2010,2014,2018,2021
POZYCJA,,,,
1,ŚWIĘTOKRZYSKIE,ŚWIĘTOKRZYSKIE,ŁÓDZKIE,POMORSKIE
2,PODLASKIE,WARMIŃSKO-MAZURSKIE,ŚWIĘTOKRZYSKIE,ŁÓDZKIE
3,WARMIŃSKO-MAZURSKIE,ŁÓDZKIE,WARMIŃSKO-MAZURSKIE,PODLASKIE
4,LUBELSKIE,ŚLĄSKIE,ŚLĄSKIE,ŚWIĘTOKRZYSKIE
5,ŁÓDZKIE,PODLASKIE,MAŁOPOLSKIE,WARMIŃSKO-MAZURSKIE
6,LUBUSKIE,MAŁOPOLSKIE,DOLNOŚLĄSKIE,ŚLĄSKIE
7,KUJAWSKO-POMORSKIE,ZACHODNIOPOMORSKIE,PODLASKIE,DOLNOŚLĄSKIE
8,MAŁOPOLSKIE,LUBELSKIE,LUBUSKIE,MAŁOPOLSKIE
9,MAZOWIECKIE,LUBUSKIE,LUBELSKIE,ZACHODNIOPOMORSKIE


Jaka jest różnica pomiędzy realnym rankingiem a naszym najlepszym?

In [58]:
for rok in [2010, 2014, 2018, 2021]:    
  print(porownanie_rankingow(rankingi[rok][metoda_suma_rang.__name__], ys[rok]))


0.328125
0.296875
0.328125
0.25


## METODY AGLOMERACYJNE

Kod do tworzenia macierzy odległości na podstawie danych (odległość euklidesowa)

In [59]:
def odleglosc_euklidesowa(obiekt_1, obiekt_2):
  square = np.vectorize(lambda x: x**2)
  return sum(square(obiekt_1 - obiekt_2))**0.5

def macierz_odleglosci(dane_df):
  std_scaler = StandardScaler()
  dane_df = pd.DataFrame(std_scaler.fit_transform(dane_df), columns = dane_df.columns, index = dane_df.index)
  D = np.identity(dane_df.shape[0])
  for obj_1_nr in range(dane_df.shape[0]):
    for obj_2_nr in range(dane_df.shape[0]):
      D[obj_1_nr][obj_2_nr] = odleglosc_euklidesowa(dane_df.iloc[obj_1_nr, :], dane_df.iloc[obj_2_nr, :])
  return pd.DataFrame(D, columns = dane_df.index, index = dane_df.index)

In [60]:

d_2010 = macierz_odleglosci(yearly_data[2010])
d_2014 = macierz_odleglosci(yearly_data[2014])
d_2018 = macierz_odleglosci(yearly_data[2018])
d_2021 = macierz_odleglosci(yearly_data[2021])

### Funkcja do lokalizowania, pomiędzy którymi grupami jest najmniejsza odległość

In [93]:
def locate_minimum(D):
  minimum = D.fillna(value=np.inf).min().min()
  nazwa = D.loc[:, (D == minimum).any()].iloc[:, 0].name
  indeks = D[nazwa].idxmin()
  return(nazwa, indeks)

### Funkcje liczące odległości według różnych metod aglomeracyjnych
Każda funkcja przyjmuje wektor odległości dla każdej grupy i ewentualny trzeci parametr w postaci wszystkich nazw grup (zagregowane podgrupy w ramach grupy oddzielane są spacjami).

In [189]:
def metoda_srodka_ciezkosci(X1, X2, columny):
  licznosc_pierwsza, licznosc_druga = X1.name.count(' ')+1, X2.name.count(' ')+1
  licznosc_calk = licznosc_pierwsza + licznosc_druga
  return licznosc_pierwsza/licznosc_calk * X1 + licznosc_druga/licznosc_calk * X2 - \
  licznosc_pierwsza * licznosc_druga /licznosc_calk**2 * min(X1)

def metoda_sredniej_grupowej(X1, X2, columny):
  licznosc_pierwsza, licznosc_druga = X1.name.count(' ')+1, X2.name.count(' ')+1
  licznosc_calk = licznosc_pierwsza + licznosc_druga
  return X1 * licznosc_pierwsza/licznosc_calk + X2 * licznosc_druga / licznosc_calk

def metoda_warda(X1, X2, columny):
  licznosci = (columny.copy().apply(lambda x: x.count(' '))+1).values
  licznosci = pd.Series(licznosci, index=columny)
  licznosc_pierwsza, licznosc_druga = X1.name.count(' ')+1, X2.name.count(' ')+1
  licznosc_calk = licznosc_pierwsza + licznosc_druga
  #print(min(X1.fillna(value=np.inf)))
  return (licznosci+licznosc_pierwsza)/(licznosci+licznosc_calk)*X1 + \
  (licznosci+licznosc_druga)/(licznosci+licznosc_calk)*X2 \
  - licznosci/(licznosci+licznosc_calk)*min(X1.fillna(value=np.inf))

def najdalszy_sasiad(X1, X2, columny=np.nan):
  return 0.5 * X1 + 0.5 * X2 + 0.5 * (X1 - X2).apply(lambda x: math.fabs(x))

def najblizszy_sasiad(X1, X2, columny=np.nan):
  return 0.5 * X1 + 0.5 * X2 - 0.5 * (X1 - X2).apply(lambda x: math.fabs(x))

def mediana(X1, X2):
  return 0.5 * X1 + 0.5 * X2 - 0.25 * X1 * X2


### Funkcja licząca jeden krok wybranej metody aglomeracyjnej
Przyjmuje w argumencie macierz odległości grup oraz nazwę funkcji wybranej metody aglomeracyjnej. Łączy ona dwie grupy o najmniejszej odległości. Jako parametr przyjmuje macierz odległości i nazwę funkcji opisującej wybraną przez nas metodę aglomeracyjną.

In [174]:
def krok_metoda_aglomeracyjna(D : pd.DataFrame, formula):
  group1, group2 = locate_minimum(D)
  #print(group1, ';;', group2)
  kolejna = D.copy()
  #print(kolejna)
  kolejna = D.drop(group2, axis=1).drop(group2, axis=0)
  kolejna = kolejna.rename(columns={group1:group1+';'+group2}, index={group1:group1+';'+group2})
  new_ = formula(D[group1], D[group2], pd.Series(D.columns))
  kolejna[group1+';'+group2] = new_.drop(group2)
  kolejna.loc[group1+';'+group2] = new_.drop(group2)
  return kolejna


Tutaj autorska funkcja wyliczająca Silhouette Coefficient

In [256]:
def inner_cluster_distance(klaster_badany, dane_oryginalne, klaster_najblizszy):
  """
  Przyjmuje w parametrze klaster w formacie OBIEKT1;OBIEKT2;OBIEKT3 <- jeden klaster
  z obiektami dla których wylicza odległość międzyklastrową
  """
  obiekty_klaster_badany = klaster_badany.split(';')
  obiekty_klaster_najblizszy = klaster_najblizszy.split(';')
  ais = pd.Series({}, dtype='float64').rename('ais')
  for obiekt1 in obiekty_klaster_badany:
    licznik, suma = 0, 0
    for obiekt2 in obiekty_klaster_badany:
      if obiekt1 == obiekt2: continue
      suma+= odleglosc_euklidesowa(dane_oryginalne.loc[obiekt1], dane_oryginalne.loc[obiekt2])
      licznik+=1
    ais[obiekt1] = suma/licznik if licznik!=0 else np.inf # chcemy wymusić klasteryzacje
    
    bis = pd.Series({}, dtype='float64').rename('bis')
    for obiekt1 in obiekty_klaster_badany:
      licznik, suma = 0, 0
      for obiekt2 in obiekty_klaster_najblizszy:
        if obiekt1 == obiekt2: continue
        suma+= odleglosc_euklidesowa(dane_oryginalne.loc[obiekt1], dane_oryginalne.loc[obiekt2])
        licznik+=1
      bis[obiekt1] = suma/licznik if licznik!=0 else np.inf # chcemy wymusić klasteryzacje
  return pd.concat([bis, ais], axis=1)
  


### Funkcja licząca dowolną ilość złączeń wybranej metody aglomeracyjnej
Jej parametry to: wyjściowa macierz odległości między obiektami, funkcję opisującą wybraną metodę aglomeracyjną oraz ilość złączeń (musi być mniejsza niż liczba obiektów). Bez podania liczby kroków, dokonane zostanie złączenie wszystkich obiektów ze sobą (licza kroków = liczba grup)

In [420]:
def iteracja_aglomeracyjna(dystanse : pd.DataFrame, formula, dane_oryginalne, ilosc_krokow=np.inf):
  dystanse = dystanse.replace(0, np.nan)
  if ilosc_krokow == np.inf or ilosc_krokow > dystanse.shape[0]:
    ilosc_krokow = dystanse.shape[0] - 2
  for krok in range(1, ilosc_krokow+1):
    dystanse = krok_metoda_aglomeracyjna(dystanse, formula)
    #wyliczam dla kazdego mozlwiego elementu jego ai i bi
    ai_bi = pd.DataFrame({})
    for cluster in dystanse.index:
      #szukamy najblizszego sasiada
      closest_cluster = dystanse.columns[np.argmin(dystanse.loc[cluster, :])] # mamy najbliższy cluster
      ai_bi = pd.concat([ai_bi, inner_cluster_distance(cluster, yearly_data[2010], closest_cluster)], axis=0)
    denominator = np.maximum(ai_bi['ais'], ai_bi['bis'])
    #print(denominator)
    silouette_score = {k:0 for k in list(ai_bi.index)}
    for index in ai_bi.index:
      if ai_bi.loc[index]['ais'] == np.inf: 
        silouette_score[index] = 0
      else:
        silouette_score[index] = (ai_bi.loc[index, 'bis'] - ai_bi.loc[index, 'ais'])/denominator.loc[index]
    print(pd.Series(silouette_score).mean())

    #print(f"KROK {krok}. ITERACJI \n --------------------------------- \n", dystanse)
  return dystanse

### TEST

In [423]:
X = iteracja_aglomeracyjna(d_2010, metoda_sredniej_grupowej, yearly_data[2010],200)

0.11270618050834985
0.21597730964658804
0.282013073362786
0.15634704319900547
0.22996004680080764
0.1961216293458094
0.1699972384311541
0.272948093405257
0.027295040806798898
-0.03844686672471729
-0.07868769800663349
0.15750690373280485
0.42169320464792504
0.5488582650797076
